<a href="https://colab.research.google.com/github/Riley-Hoang/3603-Programming-for-Data-Science/blob/main/Assignments/06-Working_with_Data_Adv/05_basic_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np

# NOTE: Creating a messy dataset that contains:
# - Missing values (None / NaN)
# - Wrong data types (text inside numeric columns)
# - Invalid dates
# - Empty strings ""
# - A new column with None and empty strings
# - One fully incorrect/broken row

data = {
    "Name": ["Alice", "Bob", "Charlie", "David", None],
    "Age": ["25", "thirty", 35, np.nan, "40"],
    "Signup Date": ["2022-01-01", "not a date", "2022/03/01", None, "April 5, 2022"],
    "Score": [95.5, None, 88.0, 92.5, ""],
}

df = pd.DataFrame(data)

# Add a new column with None and ""
df["Comments"] = ["ok", None, "", "good", ""]

# Add a completely broken row
broken_row = {
    "Name": "",
    "Age": "forty-two years",
    "Signup Date": "32/13/2022",
    "Score": "ninety nine",
    "Comments": None
}

df = pd.concat([df, pd.DataFrame([broken_row])], ignore_index=True)

print("=== Original Messy DataFrame ===")
print(df)


=== Original Messy DataFrame ===
      Name              Age    Signup Date        Score Comments
0    Alice               25     2022-01-01         95.5       ok
1      Bob           thirty     not a date         None     None
2  Charlie               35     2022/03/01         88.0         
3    David              NaN           None         92.5     good
4     None               40  April 5, 2022                      
5           forty-two years     32/13/2022  ninety nine     None


In [5]:
# NOTE: Detect missing values in the entire DataFrame
print("\n=== Detect Missing Values (True = Missing) ===")
print(df.isnull())

# NOTE: Count total missing values per column
print("\n=== Missing Values Count ===")
print(df.isnull().sum())

# NOTE: Show rows that contain at least one missing value
print("\n=== Rows Containing Missing Values ===")
print(df[df.isnull().any(axis=1)])



=== Detect Missing Values (True = Missing) ===
    Name    Age  Signup Date  Score  Comments
0  False  False        False  False     False
1  False  False        False   True      True
2  False  False        False  False     False
3  False   True         True  False     False
4   True  False        False  False     False
5  False  False        False  False      True

=== Missing Values Count ===
Name           1
Age            1
Signup Date    1
Score          1
Comments       2
dtype: int64

=== Rows Containing Missing Values ===
    Name              Age    Signup Date        Score Comments
1    Bob           thirty     not a date         None     None
3  David              NaN           None         92.5     good
4   None               40  April 5, 2022                      
5         forty-two years     32/13/2022  ninety nine     None


In [6]:
# NOTE: Example of fillna() with different default values per column
filled_df = df.fillna({
    "Name": "Unknown",
    "Age": -1,
    "Signup Date": "1970-01-01",
    "Score": 0.0,
    "Comments": "No comment"
})
print("\n=== Example: fillna with defaults ===")
print(filled_df)

# NOTE: Example of dropping rows that contain ANY null values
drop_df = df.dropna()
print("\n=== Example: dropna() removing incomplete rows ===")
print(drop_df)



=== Example: fillna with defaults ===
      Name              Age    Signup Date        Score    Comments
0    Alice               25     2022-01-01         95.5          ok
1      Bob           thirty     not a date          0.0  No comment
2  Charlie               35     2022/03/01         88.0            
3    David               -1     1970-01-01         92.5        good
4  Unknown               40  April 5, 2022                         
5           forty-two years     32/13/2022  ninety nine  No comment

=== Example: dropna() removing incomplete rows ===
      Name Age Signup Date Score Comments
0    Alice  25  2022-01-01  95.5       ok
2  Charlie  35  2022/03/01  88.0         


In [7]:
print("\n=== Data Types BEFORE Cleaning ===")
print(df.dtypes)

# NOTE: Convert Age and Score to numeric
# Any invalid data will become NaN (errors='coerce')
df["Age_clean"] = pd.to_numeric(df["Age"], errors="coerce")
df["Score_clean"] = pd.to_numeric(df["Score"], errors="coerce")

# NOTE: Convert Signup Date to datetime safely
df["Signup_clean"] = pd.to_datetime(df["Signup Date"], errors="coerce")

print("\n=== Data Types AFTER Conversion ===")
print(df[["Age_clean", "Score_clean", "Signup_clean"]].dtypes)



=== Data Types BEFORE Cleaning ===
Name           object
Age            object
Signup Date    object
Score          object
Comments       object
dtype: object

=== Data Types AFTER Conversion ===
Age_clean              float64
Score_clean            float64
Signup_clean    datetime64[ns]
dtype: object


In [10]:
# =========================
# Part 5: Impute (fixed)
# NOTE (EN): This block performs robust imputation after safe type conversion.
# - It first normalizes empty strings to NaN so they are treated as missing.
# - It computes fallbacks (median/mean/default date) when needed.
# - It fills Name/Comments consistently for both None and empty strings.
# - Works even if there are no valid numeric or date values.
# =========================

import numpy as np
import pandas as pd

# 5.0 Normalize empty strings -> NaN so "" is considered missing for all object cols
df = df.replace("", np.nan)

# (If conversion was done earlier, re-run conversions to reflect the replacement)
df["Age_clean"] = pd.to_numeric(df["Age"], errors="coerce")
df["Score_clean"] = pd.to_numeric(df["Score"], errors="coerce")
df["Signup_clean"] = pd.to_datetime(df["Signup Date"], errors="coerce")

# 5.1 Age: fill with median; fallback to -1 if median is NaN
age_median = df["Age_clean"].median()
if pd.isna(age_median):
    age_median = -1  # business decision: indicate unknown by -1
df["Age_clean"].fillna(age_median, inplace=True)

# 5.2 Score: fill with mean; fallback to 0.0 if mean is NaN
score_mean = df["Score_clean"].mean()
if pd.isna(score_mean):
    score_mean = 0.0
df["Score_clean"].fillna(score_mean, inplace=True)

# 5.3 Signup Date: fill with earliest valid date, fallback to 1970-01-01
if df["Signup_clean"].notna().any():
    earliest = df["Signup_clean"].min()
    # ensure earliest is a Timestamp, otherwise fallback
    if pd.isna(earliest):
        earliest = pd.to_datetime("1970-01-01")
else:
    earliest = pd.to_datetime("1970-01-01")
df["Signup_clean"].fillna(earliest, inplace=True)

# 5.4 Name: treat both NaN and empty/whitespace as missing -> 'Unknown'
# strip whitespace first to catch "   "
df["Name_clean"] = df["Name"].astype("string").str.strip()
df["Name_clean"].replace({"<NA>": None}, inplace=True)   # optional for pandas string NA
df["Name_clean"] = df["Name_clean"].fillna("Unknown")
df["Name_clean"].replace("", "Unknown", inplace=True)

# 5.5 Comments: fill missing comments with 'No comment'
df["Comments_clean"] = df["Comments"].astype("string").str.strip().fillna("No comment")
df["Comments_clean"].replace("", "No comment", inplace=True)

# After imputation, optional: convert types to explicit dtypes
df["Age_clean"] = df["Age_clean"].astype(float)   # or int if you prefer after handling -1
df["Score_clean"] = df["Score_clean"].astype(float)
df["Signup_clean"] = pd.to_datetime(df["Signup_clean"])

# Show the results for verification
print("\n--- After Robust Imputation ---")
print(df[["Name", "Name_clean", "Age", "Age_clean", "Score", "Score_clean", "Signup Date", "Signup_clean", "Comments", "Comments_clean"]])



--- After Robust Imputation ---
      Name Name_clean              Age  Age_clean        Score  Score_clean  \
0    Alice      Alice               25       25.0         95.5         95.5   
1      Bob        Bob           thirty       35.0         None         92.0   
2  Charlie    Charlie               35       35.0         88.0         88.0   
3    David      David              NaN       35.0         92.5         92.5   
4     None    Unknown               40       40.0          NaN         92.0   
5      NaN    Unknown  forty-two years       35.0  ninety nine         92.0   

     Signup Date Signup_clean Comments Comments_clean  
0     2022-01-01   2022-01-01       ok             ok  
1     not a date   2022-01-01     None     No comment  
2     2022/03/01   2022-01-01      NaN     No comment  
3           None   2022-01-01     good           good  
4  April 5, 2022   2022-01-01      NaN     No comment  
5     32/13/2022   2022-01-01     None     No comment  


/tmp/ipython-input-3433781500.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age_clean"].fillna(age_median, inplace=True)
/tmp/ipython-input-3433781500.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

In [11]:
# NOTE: Create a final cleaned DataFrame
clean_df = df[[
    "Name_clean", "Age_clean", "Signup_clean",
    "Score_clean", "Comments_clean"
]].rename(columns={
    "Name_clean": "Name",
    "Age_clean": "Age",
    "Signup_clean": "Signup Date",
    "Score_clean": "Score",
    "Comments_clean": "Comments"
})

print("\n=== FINAL CLEANED DATAFRAME ===")
print(clean_df)

print("\n=== CLEANED DATAFRAME INFO ===")
print(clean_df.info())

print("\n=== CLEANED DATAFRAME SUMMARY ===")
print(clean_df.describe(include="all"))



=== FINAL CLEANED DATAFRAME ===
      Name   Age Signup Date  Score    Comments
0    Alice  25.0  2022-01-01   95.5          ok
1      Bob  35.0  2022-01-01   92.0  No comment
2  Charlie  35.0  2022-01-01   88.0  No comment
3    David  35.0  2022-01-01   92.5        good
4  Unknown  40.0  2022-01-01   92.0  No comment
5  Unknown  35.0  2022-01-01   92.0  No comment

=== CLEANED DATAFRAME INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Name         6 non-null      string        
 1   Age          6 non-null      float64       
 2   Signup Date  6 non-null      datetime64[ns]
 3   Score        6 non-null      float64       
 4   Comments     6 non-null      string        
dtypes: datetime64[ns](1), float64(2), string(2)
memory usage: 372.0 bytes
None

=== CLEANED DATAFRAME SUMMARY ===
           Name        Age          Signu